### Setup

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

### Gather information 

First, let's define the part of the graph that will gather user requirements.

In [2]:
from typing import List

from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI

from pydantic import BaseModel

In [3]:
template = """Your job is to get information from a user about what type of prompt template they want to create.

You should get the following information from them:

- What the objective of the prompt is
- What variables will be passed into the prompt template
- Any constraints for what the output should NOT do
- Any requirements that the output MUST adhere to

If you are not able to discern this info, ask them to clarify! Do not attempt to wildly guess.

After you are able to discern all the information, call the relevant tool."""

In [4]:
def get_messages_info(messages):
    return [SystemMessage(content=template)] + messages

In [6]:
get_messages_info(["aa", "bb"])


[SystemMessage(content='Your job is to get information from a user about what type of prompt template they want to create.\n\nYou should get the following information from them:\n\n- What the objective of the prompt is\n- What variables will be passed into the prompt template\n- Any constraints for what the output should NOT do\n- Any requirements that the output MUST adhere to\n\nIf you are not able to discern this info, ask them to clarify! Do not attempt to wildly guess.\n\nAfter you are able to discern all the information, call the relevant tool.', additional_kwargs={}, response_metadata={}),
 'aa',
 'bb']

In [7]:
class PromptInstructions(BaseModel):
    """Instructions on how to prompt the LLM."""

    objective: str
    variables: List[str]
    constraints: List[str]
    requirements: List[str]

In [8]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tool = llm.bind_tools([PromptInstructions])

In [9]:
def info_chain(state):
    messages = get_messages_info(state["messages"])
    response = llm_with_tool.invoke(messages)
    return {"messages": [response]}


In [10]:
# info_chain({"messages": [HumanMessage(content="I want to create a prompt template that generates a list of 100 random numbers between 1 and 1000. The numbers should be unique and sorted in ascending order.")]})


### Generate Prompt